In [1]:
# Generate the metadata lookup table using Quine-McCluskey to simplify the information ROM

In [1]:
import pandas as pd
from quine_mccluskey.qm import QuineMcCluskey
from datetime import datetime

In [2]:
data = pd.read_csv("/Users/anish/Desktop/student-designs-export.csv")
data = data[~data["Project Name"].isnull()]

In [3]:
len(data)

41

In [4]:
table = {}
maxlen = 0
for _, x in data.iterrows():
    idx = int(x["Project Name"].split("_")[0].replace("d", ""))
    proj = x["Project"]
    stu = str(x["Student"])
    
    if stu == "nan":
        out = proj + " (demo)"
    else:
        out = proj + " by " + stu
        
    out += "\0"
    
    print(idx, x["Project Name"])
    print(out)
    print()
    
    if maxlen < len(out): maxlen = len(out)
    table[idx] = out
    
table[60] = f"Student projects for Spring 2023 course at Carnegie Mellon\0"
table[61] = f"18-224/18-624/98-154 Intro to Open-Source Chip Design\0"
table[62] = f"Course organized by Anish Singhani and Prof. Bill Nace\0"
table[63] = f"ROM generated {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}\0"

if len(table[60]) > maxlen: maxlen = len(table[60])

1 d01_example_adder
6-Bit Combinational Adder (demo) 

2 d02_example_counter
12-Bit Counter (demo) 

3 d03_example_beepboop
"Beep Boop" Traffic Light Controller (demo) 

4 d04_example_aes128sca
AES-128 Side-Channel Model (demo) 

5 d05_meta_info
Tapeout Meta-Info (demo) 

6 d06_demo_vgapong
Pong Game with VGA (demo) 

7 d07_demo_vgagame
Endless Runner Game with VGA (demo) 

10 d10_wabib_tictactoe
Tic-Tac-Toe Game by Wahib Abib 

11 d11_gbailey_bfchip
Brainfuck CPU by Gary Bailey 

12 d12_oball_i2c
I2C Peripheral by Owen Ball 

13 d13_jrduvall_s444
S444 FPGA Logic Cell by Jack Duvall 

14 d14_jessief_trafficlight
Traffic Light Controller by Jessie Fan 

15 d15_jerryfen_prng
Pseudo Random Number Generator by Jerry Feng 

16 d16_bgonzale_pll
Digital Phase Locked Loop by Joel Gonzalez 

17 d17_njayawar_tetris
Tetris Game by Navod Jayawardhane 

18 d18_nikhildj_mac
Multiply-Accumulate Unit by Nikhil Dinkar Joshi 

19 d19_rdkapur_encryptor
Encryption Unit (custom cipher) by Roman Kapur 

20 

In [5]:
assert maxlen < 64
len(table), maxlen

(45, 59)

In [6]:
one_terms = [[] for _ in range(8)]
dc_terms = [[] for _ in range(8)]

for ii in range(8):
    ones = one_terms[ii]
    dc = dc_terms[ii]
    
    for idx in range(64):
        for i in range(64):
            val = (idx << 6) | i

            string = table.get(idx, "\0")
            if i < len(string):
                c = ord(string[i])
                if (c & (1 << ii)) != 0:
                    ones.append(val)
            else:
                dc.append(val)


In [7]:
def convert_minterm(term, bits):
    term = term[::-1]
    s = []
    for i in range(bits):
        if term[i] == "-":
            pass
        elif term[i] == "1":
            s.append(f"index[{i}]")
        elif term[i] == "0":
            s.append(f"~index[{i}]")
        else:
            assert False
    
    if len(s) == 0:
        return "0"
    else:
        return "(" + " & ".join(s) + ")"

In [8]:
statements = ""

for i in range(8):
    print(i)
    qm = QuineMcCluskey()
    s = qm.simplify(ones=one_terms[i], dc=dc_terms[i], num_bits=12)
    mt = " | ".join([convert_minterm(x, 12) for x in s])
    statements += (f"assign val[{i}] = {mt};\n\n")
    

verilog = f"""
`default_nettype none

module my_chip (
    input logic [11:0] io_in,
    input logic clock, reset,
    output logic [11:0] io_out
);

wire [7:0] val;
reg [11:0] index;

always_ff @(posedge clock) begin
    index <= io_in;
    io_out <= {{4'b0000, val}};
end

{statements}

endmodule
"""
    
with open("src/chip.sv", "w+") as f:
    f.write(verilog)

0
1
2
3
4
5
6
7
